In [1]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from ipywidgets import interact, DatePicker
from datetime import datetime, timedelta

# Константы для URL и формата даты
CURRENCY_URL_TEMPLATE = "https://www.cbr.ru/scripts/XML_dynamic.asp?date_req1={start_date}&date_req2={end_date}&VAL_NM_RQ=R01235"
DATE_FORMAT = "%d.%m.%Y"
DATE_FORMAT_REQUEST = "%d/%m/%Y"


def fetch_currency_data(start_date, end_date):
    """
    Получает данные о курсе валюты с сайта ЦБ РФ.

    Args:
        start_date (str): Дата начала в формате DATE_FORMAT_REQUEST.
        end_date (str): Дата окончания в формате DATE_FORMAT_REQUEST.

    Returns:
        pandas.DataFrame: DataFrame с данными о курсе валюты.
                           Возвращает пустой DataFrame, если данные не получены.
    """
    url = CURRENCY_URL_TEMPLATE.format(start_date=start_date, end_date=end_date)
    try:
        response = requests.get(url)
        response.raise_for_status()  # Проверка на ошибки HTTP
        tree = ET.ElementTree(ET.fromstring(response.content))
        root = tree.getroot()

        data = []
        for record in root.findall('Record'):
            date_str = record.attrib['Date']
            value_str = record.find('Value').text.replace(',', '.')
            try:
                date = datetime.strptime(date_str, DATE_FORMAT)
                value = float(value_str)
                data.append([date, value])
            except (ValueError, TypeError):
                print(f"Ошибка при обработке записи: Дата - {date_str}, Значение - {value_str}")
                continue # Пропускаем некорректные данные


        df = pd.DataFrame(data, columns=['Date', 'USD/RUB'])
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)
        return df
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе данных: {e}")
        return pd.DataFrame()



def plot_currency(start_date, end_date):
    """
    Строит график курса валюты.

    Args:
        start_date (datetime.date): Дата начала.
        end_date (datetime.date): Дата окончания.
    """    
    plt.clf()
    fig, ax = plt.subplots(figsize=(10, 6))

    df_filtered = df.loc[start_date:end_date]

    ax.plot(df_filtered.index, df_filtered['USD/RUB'], marker='o', linestyle='-', color='blue')

    ax.set_title('Курс USD к RUB по данным ЦБ РФ')
    ax.set_xlabel('Дата')
    ax.set_ylabel('Курс')
    ax.grid(True)

    ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=1))
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%d-%m-%Y"))
    fig.autofmt_xdate() # Автоматический поворот меток дат


    plt.tight_layout() #  Для предотвращения обрезания меток
    plt.show()



# Загрузка данных. timedelta(days=365) для загрузки данных за последний год
end_date_initial = datetime.now()
start_date_initial = end_date_initial - timedelta(days=365)


start_date_str = start_date_initial.strftime(DATE_FORMAT_REQUEST)
end_date_str = end_date_initial.strftime(DATE_FORMAT_REQUEST)

df = fetch_currency_data(start_date_str, end_date_str)

if not df.empty: # Проверяем, что DataFrame не пустой, прежде чем создавать виджеты
    date_start_picker = DatePicker(description='Дата начала', value=start_date_initial.date())
    date_end_picker = DatePicker(description='Дата конца', value=end_date_initial.date())

    interact(plot_currency, start_date=date_start_picker, end_date=date_end_picker);
else:
    print("Не удалось загрузить данные для построения графика.")

interactive(children=(DatePicker(value=datetime.date(2023, 10, 23), description='Дата начала'), DatePicker(val…